# What are the top 5 zipcodes in the US that a real estate firm should invest in?

## Questions to consider from Julian: 
- how do we want to briing things together? pool zip codes together, make a general model trained on all zipcodes & tested on one of the zipcode

- we can do arima modelss & control for staionarity, but are we going to put anything else into it? are there other data points in the date we have to make our predictions better

- trainn test split --> cut off a year or two or three

- suggestions: bin zip codes? maybe by state, by urban or suburban or by metro area now maybe join the mini zip codes!! specify which zip code is which justify it! these are the largest/most urban zips in the dataset

- note: there is value in just a forecast, we can assume you are already invested in this zip code and what should they do?

- we could grouby for the metro areas

# Imports and Load in Data


In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

In [8]:
#read in data
df = pd.read_csv('zillow_data.csv')
df.head(30)

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500
5,91733,77084,Houston,TX,Houston,Harris,6,95000.0,95200.0,95400.0,...,157900,158700,160200,161900,162800,162800,162800,162900,163500,164300
6,61807,10467,New York,NY,New York,Bronx,7,152900.0,152700.0,152600.0,...,394400,400000,407300,411600,413200,414300,413900,411400,413200,417900
7,84640,60640,Chicago,IL,Chicago,Cook,8,216500.0,216700.0,216900.0,...,798000,787100,776100,774900,777900,777900,778500,780500,782800,782800
8,91940,77449,Katy,TX,Houston,Harris,9,95400.0,95600.0,95800.0,...,166800,167400,168400,169600,170900,172300,173300,174200,175400,176200
9,97564,94109,San Francisco,CA,San Francisco,San Francisco,10,766000.0,771100.0,776500.0,...,3767700,3763900,3775000,3799700,3793900,3778700,3770800,3763100,3779800,3813500


# Check Data for Null Values

In [7]:
df['City'].value_counts()

New York       114
Los Angeles     95
Houston         88
San Antonio     48
Washington      45
              ... 
Point            1
Wartburg         1
Dade City        1
Chewelah         1
Manteo           1
Name: City, Length: 7554, dtype: int64

In [3]:
df['City'].isna().value_counts()

False    14723
Name: City, dtype: int64

In [ ]:
df['State'].isna().value_counts()

In [ ]:
#will probably want to drop these nulls since the number of values thatt are missing is not a huge portion of the data, we should be able to drop them without any major issues
df['Metro'].isna().value_counts()

In [ ]:
df['CountyName'].isna().value_counts()

In [ ]:
df['SizeRank'].isna().value_counts()

In [ ]:
#we have some missing here too
df['1996-04'].isna().value_counts()

In [ ]:
df['1996-05'].isna().value_counts()

In [ ]:
df['1996-06'].isna().value_counts()

In [ ]:
# it looks like we might have the same missing values for all the years
df['1996-07'].isna().value_counts()

In [ ]:
df['2000-02'].isna().value_counts()

In [ ]:
#hmmm or maybe not 
df['2018-01'].isna().value_counts()


In [ ]:
#so now we can see we have a lot of null values for past years and the Metro column; we can go ahead and drop them since roughly 1050 values out of 13700 is not a huge portion
pd.options.display.min_rows = 272

df.isnull().sum().sort_values(ascending=False)

In [ ]:
#let's go ahead and drop all the null values 
df = df.copy()
df.dropna(inplace = True)

In [ ]:
#awesome, now that our nulls are gone we can go ahead and filter for the 10% of zipcodes with respect to size_rank 
sr_10 = df.SizeRank.quantile(q=.10)
zip_5 = df[df['SizeRank']<sr_10]
zip_5.head()

# Lets try binning things by Metro/Zip/SizeRank"?


In [ ]:
df['Metro'].value_counts()
#looks like there are over 600 unique Metro areas .... i'm not sure how we go about binning these


# Convert  Date Columns  to Date Time Objects

In [9]:
def get_datetimes(df):
    """
    Takes a dataframe:
    returns only those column names that can be converted into datetime objects 
    as datetime objects.
    NOTE number of returned columns may not match total number of columns in passed dataframe
    """
    
    
    return pd.to_datetime(df.columns.values[7:], format='%Y-%m')

In [10]:
#converting the dataframe without nulls to datetime
get_datetimes(df)

DatetimeIndex(['1996-04-01', '1996-05-01', '1996-06-01', '1996-07-01',
               '1996-08-01', '1996-09-01', '1996-10-01', '1996-11-01',
               '1996-12-01', '1997-01-01',
               ...
               '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01',
               '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01',
               '2018-03-01', '2018-04-01'],
              dtype='datetime64[ns]', length=265, freq=None)

In [11]:
 df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,334200.0,335400.0,336500.0,...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,235700.0,236900.0,236700.0,...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,210400.0,212200.0,212200.0,...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,498100.0,500900.0,503100.0,...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,77300.0,77300.0,77300.0,...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


# Reshape from Wide to Long Format

In [12]:


def melt_data(df):
    """
    Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
    Returns a long-form datetime dataframe 
    with the datetime column names as the index and the values as the 'values' column.
    
    If more than one row is passes in the wide-form dataset, the values column
    will be the mean of the values from the datetime columns in all of the rows.
    """
    
    melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

In [13]:
#first we will make individual dataframes for each zipcode area 
melt_data(df)

,value
time,
1996-04-01,118299.123063
1996-05-01,118419.044139
1996-06-01,118537.423268
1996-07-01,118653.069278
1996-08-01,118780.254312
...,...
2017-12-01,281095.320247
2018-01-01,282657.060382
2018-02-01,284368.688447


In [ ]:
#merge melted dataframe and raw/original dataframe together